In [1]:
import numpy as np
import Pk_library as PKL
import os
import glob
from tqdm.notebook import tqdm

import sys
sys.path.append('../../py/')

from cosmoSim import cosmoSim

In [2]:
outdir = '../../data_prods/'
data_dir = '/home/ryan/Data'
subdir_list = [os.path.join(data_dir, o) for o in os.listdir(data_dir) if os.path.isdir(os.path.join(data_dir,o))]

print(subdir_list)

['/home/ryan/Data/run_2cDM_L3N256_DM_powerm2m2_sigma1_dir_2', '/home/ryan/Data/run_2cDM_L3N256_DM_power00_sigma1_dir_7_Vkick119.90', '/home/ryan/Data/run_2cDM_L3N256_DM_powerm2m2_sigma0.1', '/home/ryan/Data/run_2cDM_L3N256_DM_powerm2m2_sigma5.0_dir_9', '/home/ryan/Data/run_2cDM_L3N512_HY_power00_sigma1', '/home/ryan/Data/run_2cDM_L3N256_HY_power00_sigma0.1', '/home/ryan/Data/run_2cDM_L25N256_DM_power00_sigma1_dir_4_Vkick55.65', '/home/ryan/Data/run_CDM_L5N256_DM', '/home/ryan/Data/run_2cDM_L3N256_HY_power00_sigma1_dir_9', '/home/ryan/Data/run_SIDM_L3N256_DM_power0_sigma1', '/home/ryan/Data/run_2cDM_L3N256_HY_powerm2m2_sigma1_dir_5', '/home/ryan/Data/run_2cDM_L25N256_DM_power00_sigma1_dir_7_Vkick119.90', '/home/ryan/Data/run_2cDM_L3N256_DM_powerm2m2_sigma1_dir_1', '/home/ryan/Data/lost+found', '/home/ryan/Data/run_2cDM_L3N256_HY_power00_sigma1_dir_5', '/home/ryan/Data/run_2cDM_L3N256_DM_power00_sigma1_dir_3', '/home/ryan/Data/run_2cDM_L3N256_HY_powerm2m2_sigma10', '/home/ryan/Data/run_2

In [3]:
def calculate_pk(snap, out, part_types=[1], grid_size=256):  

    # parameters
    snapshot      = snap #snapshot name
    grid          = grid_size    #grid size
    particle_type = part_types #use dark matter [1]
    do_RSD        = False   #move particles to redshift-space and calculate Pk in redshift-space
    axis          = 1      #RSD placed along the y-axis
    cpus          = 8      #number of openmp threads
    folder_out    = out #folder where to write results
    # compute power spectrum of the snapshot

    PKL.Pk_Gadget(snapshot, grid, particle_type, do_RSD, axis, cpus, folder_out)



In [4]:
rewrite = False
for subdir in tqdm(subdir_list):
    
    #run name is just folder name
    run_name = subdir.split('/')[-1]

    #reject folders that aren't runs
    if 'run' not in run_name:
        continue

    fpath = os.path.join(outdir, run_name)
    try:
        os.mkdir(fpath)
    except:
        print(f'{fpath} already exists!')

    snaps = sorted(glob.glob(subdir + '/snap*'))
    
    if not rewrite:
        if (len(glob.glob(fpath + "/Pk_CDM_z=*")) == len(snaps)):
            print(f'PK files exist for {run_name}.')
            print('Skipping...')
            continue
          
        
    fsize = os.stat(snaps[-1]).st_size / (1024**3)
    if fsize > 20:
        print(f'File size too big for {run_name}...')
        continue
    '''
    Run pylians here
    '''

    run = cosmoSim(run_name)
    grid_size = run.npart
    print('Running pylians power spectra for ', run_name)
    for i, snap in enumerate(snaps):
        print(f"Working on {snap}")
        part_types = [1]
        if run.baryon_type == "HY":
            part_types = [0, 1, 4]
        try:
            calculate_pk(snap, fpath, part_types, grid_size)
        except Exception as e:
            print(e)
            if run.baryon_type == "HY":
                part_types = [0, 1]
            try:
                calculate_pk(snap, fpath, part_types, grid_size)
            except Exception as e:
                part_types = [1]
                calculate_pk(snap, fpath, part_types, grid_size)

    

  0%|          | 0/169 [00:00<?, ?it/s]

../../data_prods/run_2cDM_L3N256_DM_powerm2m2_sigma1_dir_2 already exists!
genPK files exist for run_2cDM_L3N256_DM_powerm2m2_sigma1_dir_2.
Skipping...
../../data_prods/run_2cDM_L3N256_DM_power00_sigma1_dir_7_Vkick119.90 already exists!
genPK files exist for run_2cDM_L3N256_DM_power00_sigma1_dir_7_Vkick119.90.
Skipping...
../../data_prods/run_2cDM_L3N256_DM_powerm2m2_sigma0.1 already exists!
genPK files exist for run_2cDM_L3N256_DM_powerm2m2_sigma0.1.
Skipping...
../../data_prods/run_2cDM_L3N256_DM_powerm2m2_sigma5.0_dir_9 already exists!
genPK files exist for run_2cDM_L3N256_DM_powerm2m2_sigma5.0_dir_9.
Skipping...
../../data_prods/run_2cDM_L3N512_HY_power00_sigma1 already exists!
File size too big for run_2cDM_L3N512_HY_power00_sigma1...
../../data_prods/run_2cDM_L3N256_HY_power00_sigma0.1 already exists!
Running pylians power spectra for  run_2cDM_L3N256_HY_power00_sigma0.1
Working on /home/ryan/Data/run_2cDM_L3N256_HY_power00_sigma0.1/snap_000.hdf5

READING SNAPSHOTS PROPERTIES


/home/ryan/projects/Medvedev/dm_sims/jupyter/snaps_reqd/../../py/cosmoSim.py:57: UserWarning: Vkick not explicitly set in run run_2cDM_L3N256_HY_power00_sigma0.1! Assuming 100 km/s...
  warnings.warn(f'Vkick not explicitly set in run {self.run_name}! Assuming 100 km/s...')


Problem reading the block MASS

READING SNAPSHOTS PROPERTIES
Computing power spectrum...
0.000 < X [Mpc/h] < 3.000
0.000 < Y [Mpc/h] < 3.000
0.000 < Z [Mpc/h] < 3.000


Computing power spectrum of the field...
Time to complete loop = 1.01
Time taken = 1.13 seconds
Working on /home/ryan/Data/run_2cDM_L3N256_HY_power00_sigma0.1/snap_001.hdf5

READING SNAPSHOTS PROPERTIES
Omega_gas    =  0.0472789370207158
Omega_cdm    =  0.26272230461958845
Omega_nu     =  0.0
Omega_star   =  1.3667159398893185e-05
Omega_m      =  0.3100149087997031
Omega_m snap =  0.301712

Computing the auto- and cross-power spectra of types:  0 - 1
saving results in:
../../data_prods/run_2cDM_L3N256_HY_power00_sigma0.1/Pk_GAS_z=8.955.dat 
 ../../data_prods/run_2cDM_L3N256_HY_power00_sigma0.1/Pk_CDM_z=8.955.dat 
 ../../data_prods/run_2cDM_L3N256_HY_power00_sigma0.1/Pk_GCDM_z=8.955.dat

Computing power spectra of the fields...
Time FFTS = 0.23
Time loop = 0.94
Time taken = 1.18 seconds

Computing the auto- and cross-pow

/home/ryan/projects/Medvedev/dm_sims/jupyter/snaps_reqd/../../py/cosmoSim.py:57: UserWarning: Vkick not explicitly set in run run_2cDM_L3N256_HY_power00_sigma1_dir_9! Assuming 100 km/s...
  warnings.warn(f'Vkick not explicitly set in run {self.run_name}! Assuming 100 km/s...')


...

Computing power spectrum...
0.000 < X [Mpc/h] < 3.000
0.000 < Y [Mpc/h] < 3.000
0.000 < Z [Mpc/h] < 3.000


Computing power spectrum of the field...
Time to complete loop = 0.12
Time taken = 0.14 seconds
Working on /home/ryan/Data/run_2cDM_L3N128_HY_power00_sigma0.1_var6/snap_001.hdf5

READING SNAPSHOTS PROPERTIES
Omega_gas    =  0.04728278963353948
Omega_cdm    =  0.26272230461958845
Omega_nu     =  0.0
Omega_star   =  9.814545356052797e-06
Omega_m      =  0.310014908798484
Omega_m snap =  0.301712

Computing the auto- and cross-power spectra of types:  0 - 1
saving results in:
../../data_prods/run_2cDM_L3N128_HY_power00_sigma0.1_var6/Pk_GAS_z=8.955.dat 
 ../../data_prods/run_2cDM_L3N128_HY_power00_sigma0.1_var6/Pk_CDM_z=8.955.dat 
 ../../data_prods/run_2cDM_L3N128_HY_power00_sigma0.1_var6/Pk_GCDM_z=8.955.dat

Computing power spectra of the fields...
Time FFTS = 0.02
Time loop = 0.11
Time taken = 0.13 seconds

Computing the auto- and cross-power spectra of types:  0 - 4
saving results

/home/ryan/projects/Medvedev/dm_sims/jupyter/snaps_reqd/../../py/cosmoSim.py:57: UserWarning: Vkick not explicitly set in run run_2cDM_L3N256_HY_powerm2m2_sigma1_dir_0! Assuming 100 km/s...
  warnings.warn(f'Vkick not explicitly set in run {self.run_name}! Assuming 100 km/s...')


Problem reading the block MASS

READING SNAPSHOTS PROPERTIES
Computing power spectrum...
0.000 < X [Mpc/h] < 3.000
0.000 < Y [Mpc/h] < 3.000
0.000 < Z [Mpc/h] < 3.000


Computing power spectrum of the field...
Time to complete loop = 1.09
Time taken = 1.21 seconds
Working on /home/ryan/Data/run_2cDM_L3N256_HY_powerm2m2_sigma1_dir_0/snap_001.hdf5

READING SNAPSHOTS PROPERTIES
Omega_gas    =  0.04728552483880838
Omega_cdm    =  0.26272230461958845
Omega_nu     =  0.0
Omega_star   =  7.079340842192353e-06
Omega_m      =  0.31001490879923904
Omega_m snap =  0.301712

Computing the auto- and cross-power spectra of types:  0 - 1
saving results in:
../../data_prods/run_2cDM_L3N256_HY_powerm2m2_sigma1_dir_0/Pk_GAS_z=8.955.dat 
 ../../data_prods/run_2cDM_L3N256_HY_powerm2m2_sigma1_dir_0/Pk_CDM_z=8.955.dat 
 ../../data_prods/run_2cDM_L3N256_HY_powerm2m2_sigma1_dir_0/Pk_GCDM_z=8.955.dat

Computing power spectra of the fields...
Time FFTS = 0.22
Time loop = 0.96
Time taken = 1.19 seconds

Computin

/home/ryan/projects/Medvedev/dm_sims/jupyter/snaps_reqd/../../py/cosmoSim.py:57: UserWarning: Vkick not explicitly set in run run_2cDM_L3N256_HY_power00_sigma1_dir_7! Assuming 100 km/s...
  warnings.warn(f'Vkick not explicitly set in run {self.run_name}! Assuming 100 km/s...')


Problem reading the block MASS

READING SNAPSHOTS PROPERTIES
Computing power spectrum...
0.000 < X [Mpc/h] < 3.000
0.000 < Y [Mpc/h] < 3.000
0.000 < Z [Mpc/h] < 3.000


Computing power spectrum of the field...
Time to complete loop = 1.05
Time taken = 1.16 seconds
Working on /home/ryan/Data/run_2cDM_L3N256_HY_power00_sigma1_dir_7/snap_001.hdf5

READING SNAPSHOTS PROPERTIES
Omega_gas    =  0.0472829139142935
Omega_cdm    =  0.26272230461958845
Omega_nu     =  0.0
Omega_star   =  9.690266125131201e-06
Omega_m      =  0.3100149088000071
Omega_m snap =  0.301712

Computing the auto- and cross-power spectra of types:  0 - 1
saving results in:
../../data_prods/run_2cDM_L3N256_HY_power00_sigma1_dir_7/Pk_GAS_z=8.955.dat 
 ../../data_prods/run_2cDM_L3N256_HY_power00_sigma1_dir_7/Pk_CDM_z=8.955.dat 
 ../../data_prods/run_2cDM_L3N256_HY_power00_sigma1_dir_7/Pk_GCDM_z=8.955.dat

Computing power spectra of the fields...
Time FFTS = 0.22
Time loop = 1.02
Time taken = 1.25 seconds

Computing the auto